In [1]:
import scispacy
import spacy
import joblib
import re
import string
import re
from nltk import word_tokenize
from collections import Counter
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold, train_test_split
from scipy.sparse import coo_matrix, hstack

from sklearn.naive_bayes import MultinomialNB
from sklearn import svm

import matplotlib.pyplot as plt
import seaborn as sns

import torch
from transformers import BertTokenizer, BertModel
import gensim.downloader as api
from gensim.models import KeyedVectors

import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [2]:
df=pd.read_csv("df (2).csv")

In [3]:
unique_categories_num = df['target'].value_counts()
counts_df_full = pd.DataFrame({'Category': unique_categories_num.index, 'Count': unique_categories_num.values})
counts_df_full

,Category,Count
0,SICK PERSON,126
1,INJURED FROM A FALL,93
2,Sick Person,57
3,LIFT ASSIST,50
4,DIFFICULTY BREATHING,44
...,...,...
148,OPEN DOOR,1
149,Fire-Commercial,1
150,MISSING PERSON,1
151,DOG BITE,1


In [4]:
df['target'] = df['target'].str.lower()
df['New Category'] = df['target'].apply(lambda x: 'Fire' if 'fire' in x.lower() else ('cardiac_arrest' if 'cardiac' in x.lower() else ('heart_attack' if 'heart' in x.lower() else x)))
df['New Category'] = df['New Category'].apply(lambda x: 'heart_attack' if 'chest pain' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'stroke' if 'stroke' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'Major_trauma' if 'injured' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'Major_trauma' if 'injury' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'seizure' if 'convulsion/seizures' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'difficulty breathing' if 'overdose / poisoning' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'difficulty breathing' if 'overdose - poisoning' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'difficulty breathing' if 'choking' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'Major_trauma' if 'fall' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'difficulty breathing' if 'overdose/poisoning' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: '911 cellular hang-up/open/misdial' if '911 hang-up/open/misdial' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: '911 cellular hang-up/open/misdial' if 'duplicate call' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'suspicious person' if 'suspicious person/vehicle' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'heart_attack' if 'cardiac_arrest' in x.lower() else x)

In [5]:
df['New Category'] = df['New Category'].apply(lambda x: 'heart_attack' if 'stroke' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'others' if x != 'heart_attack' else x)

In [6]:
df1 = pd.DataFrame(
    dict(
        ntext = df['text'],
        text=df["clean_text"],
        target= df["New Category"],
        key_words_chemical= df["key_words_chemical"],
        key_words_disease	= df["key_words_disease"]))

In [7]:
df1

,ntext,text,target,key_words_chemical,key_words_disease
0,0002.0s 0002.5s SPEAKER_01: Umm\n 0003.3s 000...,umm need dont care tanya okay number 740971896...,heart_attack,-,pain pain -
1,"0012.5s 0014.7s SPEAKER_01: 911, what's the a...",911 whats address emergency hi located forgot ...,others,smelling smoke smoke smoke smoke smoke -,-
2,"\n 0011.3s 0013.9s SPEAKER_01: 911, what's th...",911 whats location emergency would 85 harrison...,others,-,-
3,"0011.1s 0012.9s SPEAKER_01: 911, let's addres...",911 lets address emergency yeah 8681 kirkland ...,others,shoot -,pain stomachache bleeding bleeding bleeding bl...
4,0005.9s 0006.4s SPEAKER_00: now.\n 0009.6s 00...,911 whats address emergency husband fallen can...,others,-,-
...,...,...,...,...,...
1336,0004.1s 0004.7s SPEAKER_00: Thank you.\n 0009...,thank 911 would say emergency yes 53 burch roa...,others,-,-
1337,"0011.4s 0013.8s SPEAKER_02: 911, what is the ...",911 address emergency emergency 8420 todd stre...,heart_attack,-,-
1338,"0011.8s 0013.6s SPEAKER_00: 911, what's the a...",911 whats address emergency really emergency g...,others,-,-
1339,"0011.8s 0013.6s SPEAKER_00: 911, what's the a...",911 whats address emergency really emergency g...,others,-,-


In [8]:
heart_attack_df = df1[df1['target'] == 'heart_attack']

In [9]:
heart_attack_df

,ntext,text,target,key_words_chemical,key_words_disease
0,0002.0s 0002.5s SPEAKER_01: Umm\n 0003.3s 000...,umm need dont care tanya okay number 740971896...,heart_attack,-,pain pain -
11,0010.9s 0013.9s SPEAKER_01: 9-1-9 on the asse...,919 assessing emergency hi send ambulance 7556...,heart_attack,aspirin -,lightheadedness pain pain -
14,"\n 0010.4s 0012.9s SPEAKER_01: 911, what is t...",911 address emergency 3629 chapman road 3629 c...,heart_attack,-,stroke stroke stroke numbness seizure seizure ...
18,0005.1s 0006.3s SPEAKER_00: Can they get by?\...,get car get 911 whats address emergency husban...,heart_attack,aspirin aspirin aspirin viagra cialis viagra c...,pain pain -
24,\n 0011.7s 0013.6s SPEAKER_00: 911 what's add...,911 whats addressing emergency grandma whats s...,heart_attack,-,pains pain pain pain pain pain pain pain -
...,...,...,...,...,...
1271,0002.5s 0002.5s SPEAKER_00: We'll talk to you...,well talk soon 911 address emergency hi addres...,heart_attack,-,pain pain pain pain pain pain pain pain pain -
1277,0004.3s 0004.5s SPEAKER_02: Thank you.\n 0010...,thank 911 whats address emergency 716 bellevue...,heart_attack,aspirin tramadol aspirin viagra cialis tramadol -,pains pain pain aneurysm pain pain pain pain p...
1291,"0012.3s 0015.2s SPEAKER_01: 911, what is the ...",911 location emergency 30 ravine ridge drive d...,heart_attack,-,-
1327,0010.1s 0011.8s SPEAKER_00: Where is your eme...,emergency hi alan geithard 7383 winfield drive...,heart_attack,madeline -,stroke numbness -


In [10]:
ter=pd.read_csv("terminology.csv")

In [11]:
ter

,MEDICAL TERM,LAY TERMINOLOGY
0,<,Less than
1,>,Greater than
2,Abdomen,"Belly, stomach"
3,Abdominal,Referring to the body cavity which contains st...
4,Ablation,Remove
...,...,...
667,Washout period,Wait until a particular medication is no longe...
668,WBC,White blood cell
669,We are required to abide,We must
670,We may use and disclose medical information ab...,We may share your medical information to run t...


In [12]:
ter['LAY TERMINOLOGY '] = ter['LAY TERMINOLOGY '].str.split(', ')
ter = ter.explode('LAY TERMINOLOGY ')
ter= ter.reset_index(drop=True)

In [13]:
ter

,MEDICAL TERM,LAY TERMINOLOGY
0,<,Less than
1,>,Greater than
2,Abdomen,Belly
3,Abdomen,stomach
4,Abdominal,Referring to the body cavity which contains st...
...,...,...
882,Washout period,Wait until a particular medication is no longe...
883,WBC,White blood cell
884,We are required to abide,We must
885,We may use and disclose medical information ab...,We may share your medical information to run t...


In [14]:
df['clean_text'][11]

'919 assessing emergency hi send ambulance 7556 wayside avenue please trying get uncle urgent care cant even get car okay okay whats phone number chromation going instructor 7409416126 send name uh julie white okay tell exactly happened okay tell exactly happened okay covid month ago last night heart started beating really fast like almost passed really lightheaded tried get hold doctor got video chat doctor said take urgent care right like hes like weak like wants face okay anyimmigrant want heartbeating fact heart beating fast chill lightheadedness okay partners going go ahead get medic started questions currently conscious partners going go ahead get medic started questions currently conscious yeah like barely okay old im sorry old 48 okay anybody whos chest pain okay history heart problems think past oh love nah love nah love nah love nah okay come suddenly gradually see next time got recovered covid completely literally hit like completely felt weak yesterday morning almost passed

In [15]:
medical_dict = dict(zip(ter['LAY TERMINOLOGY '], ter['MEDICAL TERM ']))

In [16]:
import pandas as pd
import nltk
from nltk.util import bigrams, trigrams
from nltk.tokenize import word_tokenize


In [17]:
paragraph = df['clean_text'][11]

In [18]:
tokens = word_tokenize(paragraph)

# Generate bigrams and trigrams
bigrams_list = list(bigrams(tokens))
trigrams_list = list(trigrams(tokens))

# List to store the changed words
changed_words = []

# Check if any bigrams match the layman terms and replace them with medical terms
for bigram in bigrams_list:
    bigram_str = " ".join(bigram)
    medical_term = medical_dict.get(bigram_str.lower())
    if medical_term:
        paragraph = paragraph.replace(bigram_str, f"[{medical_term}]")
        changed_words.extend(bigram)

# Check if any trigrams match the layman terms and replace them with medical terms
for trigram in trigrams_list:
    trigram_str = " ".join(trigram)
    medical_term = medical_dict.get(trigram_str.lower())
    if medical_term:
        paragraph = paragraph.replace(trigram_str, f"[{medical_term}]")
        changed_words.extend(trigram)

# Check if any unigrams match the layman terms and replace them with medical terms
for token in tokens:
    medical_term = medical_dict.get(token.lower())
    if medical_term:
        paragraph = paragraph.replace(token, f"[{medical_term}]")
        changed_words.append(token)

# Print the updated paragraph with medical terms and highlighted words
print(paragraph)

# Print the list of changed words
print("\nChanged Words:", changed_words)


919 assessing emergency hi send ambulance 7556 wayside avenue please trying get uncle urgent care cant even get car okay okay whats phone number chromation going instructor 7409416126 send name uh julie white okay tell exactly happened okay tell exactly happened okay covid month ago last night heart started beating really fast like almost passed really lightheaded tried get hold doctor got video chat doctor said take urgent care right like hes like weak like wants face okay anyimmigrant want heartbeating fact [Palpitation  ] chill lightheadedness okay partners going go ahead get medic started questions currently conscious partners going go ahead get medic started questions currently conscious yeah like barely okay old im sorry old 48 okay anybody whos chest pain okay history heart problems think past oh love nah love nah love nah love nah okay come suddenly gradually see next time got recovered covid completely literally hit like completely felt weak yesterday morning almost passed goi